<a href="https://colab.research.google.com/github/lisernia/ITBA-Trabajo_Final_CNN_Style_Transfer/blob/master/Trabajo_Final_CNN_Style_Transfer_15_06_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [0]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
La sintaxis del comando no es correcta.


In [0]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

In [0]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("C:/Users/Damian/Pictures/cabaña.jpg")
style_reference_image_path = Path("C:/Users/Damian/Pictures/picasso-picasso-cusica-plus.jpg")
result_prefix = Path("C:/Users/Damian/Pictures/output")
iterations = 100

base_image_path





WindowsPath('C:/Users/Damian/Pictures/cabaña.jpg')

In [0]:
import tensorflow as tf
tf.test.is_gpu_available(cuda_only=True) 

True

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:
total_variation_weight:
Es el peso que se le da para total variation loss, la cual mira la coherencia visual de la imagen midiendo la continuidad espacial entre los píxeles de la imagen creada entre la imagen de estilo y contenido.

 
style_weight:
Es el peso para la style loss, la loss aprovecha múltiples capas convolucionales en el CNN (en lugar de uno) para extraer patrones significativos y representaciones capturando información pertinente a apariencia o estilo de la imagen de estilo de referencia en todas las escalas espaciales, independientemente del contenido de la imagen.



 
content_weight = El peso para contect loss.
Podemos definir la loss como la diferencia entre las  características de la imagen base y las características de la imagen combinada, manteniendo la imagen generada lo suficientemente cerca de la original.









In [0]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [0]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)
img_ncols

300

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

Load_img> Carga la imagen como una imagen a un objeto de la libreria de python con el formato que se guardo en el path y canales RGB con el parametro target_size le cambia el tamaño de acuerdo a los valores en  img_nrows y img_ncols.

img_to_array> se usa para  convertir una imagen cargada en formato PIL en un array NumPy.  

expand_dims> agrega una dimensión mas que indica la cantidad de imágenes a procesar.

preprocess_input> Se necesita preprocesar la imagen de entrada ya que este modelo espera algo ligeramente diferente a la imagen de entrada,  esta funcion extrae de los canales la media cuyos valores son 103.939, 116.779, and 123.68 los cuales  vienen del análisis de ImageNet, por lo tanto la media de los canales es cero  y cambia   de RGB a BGR en este formarto los canales de color estan ordenados como azul, verde y rojo que es el estandar para la libreria OpenCv. 


In [0]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img




# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

Basicamente es el proceso inverso al metodo preprocess, por lo que tenemos que devolver los valores de la media en este proceso y luego convertir de BGR a RGB. Tenemos que estar seguros que nuestros valores no tienen signo y son de 8 bit enteros cuyos valores están entre 0 - 255 esto se logra con la función np.clip()  

 
  


In [0]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [0]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image_path
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [0]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [0]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [0]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

 
La matriz de gram captura el estilo the los mapas de caracteristicas de la imagen de estilo de referencia y la generada.
Se utiliza para calcular la loss de estilo. Compara la matriz de gram de la imagen de entrada y la generada usando MSE. La representación del estilo computa la correlación entre diferentes filtros.  es el producto de puntos de la matriz de estilo con su transpuesta dividido por  la cantidad de elementos.
Para poder extraer el estilo de una imagen.  En lugar de usar la activación directamente, transformaremos esas activaciones en matrices de gram,  La razón por la que hacemos eso es porque las matrices de gram  capturan el estilo mejor que las activaciones directamente.
Podemos  capturar  el estilo de una imagen podemos obtener la representación mirando la correlación de los valores entre las activaciones o las características.
Por razones de performance las operaciones en una red neuronal se realizan con matrices, por lo que la correlación es una matriz que se llama gram matrix  o matriz de estilo.

- ¿Por qué se permutan las dimensiones de x?

Para poder multiplicar la matriz.  Multiplicamos la matriz por su transpuesta para convertir las filas en columnas.
 


In [0]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:
style_loss:

La Loss  se define como la diferencia de correlación presente entre los mapas de características calculados por la imagen generada y la imagen de estilo.  
El objetivo es calcular una matriz de estilo  para la imagen generada y la imagen de estilo. Luego, la loss de estilo se define como la diferencia cuadrática media entre las dos matrices de estilo. La matriz de Gram esencialmente captura la "distribución de características" de un conjunto de mapas de características en una capa dada. Al tratar de minimizar la loss de estilo entre dos imágenes, básicamente está haciendo coincidir la distribución de características entre las dos imágenes.

content_loss:
Combination: La representación de las características en la capa para la imagen generada.
Base: representación de las características en la en la capa para la  imagen original.
La loss mide la distancia entre estas características imagen generada y base. Tratando de dejar la imagen generada cerca de la original.

total_variation_loss:
Mide la coherencia de la imagen viendo la continuidad de los pixeles de las imágenes combinadas.



In [0]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [0]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [0]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [0]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [0]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:


Fmin_l_bfgs_b es una función que se usa para optimizar la loss, la cual solo se puede aplicar para vectores de una dimensión y la función de loss y gradiente se deben pasar en dos funciones separadas, por eso se construye la clase Evaluator para calcular los valores del el gradiente y la loss en una pasada ya que calcularlas de forma separada sería ineficiente. 
La tercer celda muestra cómo se evalúan la loss y el gradiente en cada iteración en este caso 100, se puede ver en cada una la imagen generada y cómo va transformando el estilo en cada ejecución.

La diferencia es que en el paper usa para optimizar la loss el gradient descent  con back-propagation y en la note usan esa funcion fmin_l_bfgs_b
El optimizador bfgs tiene una limitación que es que la función solo se puede aplicar a vectores de una dimensión las imágenes son de 3 dimensiones, se realiza una clase Evaluator basada en patrones ya  que la los valores para la función loss y el gradiente se deben pasar de forma separada.

Se podría utilizar ADAM , pero el  algoritmo L-BFGS (un algoritmo cuasi-Newton que es significativamente más rápido para converger que el  Gradient descent)  a diferencia de un clasificador, en este caso, el optimizador no recibe varias muestras diferentes y trata de generalizar  todas. En la transferencia de estilo, el optimizador recibe la misma imagen varias veces. Además, l-bfgs determina la dirección ideal y la distancia a cubrir haciendo una búsqueda de línea. En problemas estocásticos, como clasificaciones y regresiones, es un enfoque computacional costoso, sin embargo, es un buen enfoque para la transferencia de estilos. De esta manera, l-bfgs aprende más rápido que Adam en el problema en cuestión.



In [0]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [0]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [0]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 221887730000.0
Image saved as C:\Users\Damian\Pictures\output\output_at_iteration_0.png
Iteration 0 completed in 2s
Start of iteration 1
Current loss value: 89240490000.0
Image saved as C:\Users\Damian\Pictures\output\output_at_iteration_1.png
Iteration 1 completed in 2s
Start of iteration 2
Current loss value: 54349824000.0
Image saved as C:\Users\Damian\Pictures\output\output_at_iteration_2.png
Iteration 2 completed in 2s
Start of iteration 3
Current loss value: 39850164000.0
Image saved as C:\Users\Damian\Pictures\output\output_at_iteration_3.png
Iteration 3 completed in 2s
Start of iteration 4
Current loss value: 32210401000.0
Image saved as C:\Users\Damian\Pictures\output\output_at_iteration_4.png
Iteration 4 completed in 2s
Start of iteration 5
Current loss value: 26483534000.0
Image saved as C:\Users\Damian\Pictures\output\output_at_iteration_5.png
Iteration 5 completed in 2s
Start of iteration 6
Current loss value: 23235453000.0
Image sa

In [0]:
x = preprocess_image(base_image_path)
x


array([[[[ -15.939003 ,    6.2210007,   33.32     ],
         [  10.060997 ,   22.221    ,   47.32     ],
         [   5.060997 ,   14.221001 ,   35.32     ],
         ...,
         [  20.060997 ,   46.221    ,   67.32     ],
         [  21.060997 ,   47.221    ,   68.32     ],
         [  23.060997 ,   47.221    ,   68.32     ]],

        [[   8.060997 ,   23.221    ,   47.32     ],
         [   5.060997 ,   20.221    ,   44.32     ],
         [   9.060997 ,   20.221    ,   43.32     ],
         ...,
         [  20.060997 ,   45.221    ,   68.32     ],
         [  21.060997 ,   46.221    ,   69.32     ],
         [  20.060997 ,   46.221    ,   67.32     ]],

        [[  -1.939003 ,    9.221001 ,   30.32     ],
         [  -3.939003 ,    9.221001 ,   32.32     ],
         [   4.060997 ,   15.221001 ,   36.32     ],
         ...,
         [  20.060997 ,   45.221    ,   68.32     ],
         [  21.060997 ,   46.221    ,   69.32     ],
         [  18.060997 ,   45.221    ,   66.32     ]],

In [0]:
base_image_path


WindowsPath('C:/Users/Damian/Pictures/cabaña.jpg')

In [0]:
x = preprocess_image(base_image_path)
x

array([[[[ -15.939003 ,    6.2210007,   33.32     ],
         [  10.060997 ,   22.221    ,   47.32     ],
         [   5.060997 ,   14.221001 ,   35.32     ],
         ...,
         [  20.060997 ,   46.221    ,   67.32     ],
         [  21.060997 ,   47.221    ,   68.32     ],
         [  23.060997 ,   47.221    ,   68.32     ]],

        [[   8.060997 ,   23.221    ,   47.32     ],
         [   5.060997 ,   20.221    ,   44.32     ],
         [   9.060997 ,   20.221    ,   43.32     ],
         ...,
         [  20.060997 ,   45.221    ,   68.32     ],
         [  21.060997 ,   46.221    ,   69.32     ],
         [  20.060997 ,   46.221    ,   67.32     ]],

        [[  -1.939003 ,    9.221001 ,   30.32     ],
         [  -3.939003 ,    9.221001 ,   32.32     ],
         [   4.060997 ,   15.221001 ,   36.32     ],
         ...,
         [  20.060997 ,   45.221    ,   68.32     ],
         [  21.060997 ,   46.221    ,   69.32     ],
         [  18.060997 ,   45.221    ,   66.32     ]],

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:
Total_variation_weight:
Este peso nos permite balancear entre el contenido y el estilo de la imagen generada, si es muy algo en comparación a los otros pesos se pierde calidad en la imagen hay más ruido en la imagen ya   los colores de los objetos vecinos en la imagen se mezclan y la imagen es de baja calidad. Pero en una proporción adecuada ayuda a ver mejor calidad la imagen.
 
style_weight : Sí aumentó el peso del estilo se refleja más la imagen de estilo que la de contenido.
Content_weight:  Si aumento el peso de contenido la imagen se parece más a la original.
Si todos los pesos tiene cero es igual a la imagen de contenido.

las imagines se  adjunta en el zip "Respuesta 8" dentro del git.

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:
las imagines se  adjunta en el zip "Respuesta 9" dentro del git.
